In [15]:
import leafmap.foliumap as leafmap
import rasterio
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import os
import glob
import datetime
from datetime import datetime
import numpy as np
import rasterio
import numpy as np

In [16]:
omi_data = os.path.join(os.getcwd(), "omi-so2")
omps_data = os.path.join(os.getcwd(), "omps-inputs")

### Load OMI Tiles

In [29]:
#omi_input_16 = os.path.join(omi_data,'OMI-Aura_L3-OMSO2e_ColumnAmountSO2_2022_1_16_8bit.tif')
#omi_input_30 = os.path.join(omi_data,'OMI-Aura_L3-OMSO2e_ColumnAmountSO2_2022_1_30_8bit.tif')
omi_input_16 = 'https://github.com/Rhys-L/QOS_HTHH/blob/main/omi-so2/OMI-Aura_L3-OMSO2e_ColumnAmountSO2_2022_1_16_8bit.tif'
omi_input_30 = 'https://github.com/Rhys-L/QOS_HTHH/blob/main/omi-so2/OMI-Aura_L3-OMSO2e_ColumnAmountSO2_2022_1_30_8bit.tif'

#omi_mask_bright_16 = os.path.join(omi_data,'mask_OMI-Aura_L3-OMSO2e_ColumnAmountSO2_2022_1_16_8bit_brightest_multimask_false_36.tif')
#omi_mask_bright_30 = os.path.join(omi_data,'mask_OMI-Aura_L3-OMSO2e_ColumnAmountSO2_2022_1_30_8bit_brightest_multimask_false_36.tif')
omi_mask_bright_16 = 'https://github.com/Rhys-L/QOS_HTHH/blob/main/omi-so2/mask_OMI-Aura_L3-OMSO2e_ColumnAmountSO2_2022_1_16_8bit_brightest_multimask_false_36.tif'
omi_mask_bright_30 =  'https://github.com/Rhys-L/QOS_HTHH/blob/main/omi-so2/mask_OMI-Aura_L3-OMSO2e_ColumnAmountSO2_2022_1_30_8bit_brightest_multimask_false_36.tif'

#omi_mask_calipso_16 = os.path.join(omi_data,'mask_OMI-Aura_L3-OMSO2e_ColumnAmountSO2_2022_1_16_8bit_calipso_rev2_lon_lat87.tif')
#omi_mask_calipso_30 = os.path.join(omi_data,'mask_OMI-Aura_L3-OMSO2e_ColumnAmountSO2_2022_1_30_8bit_calipso_rev2_constraint1297.tif')
omi_mask_calipso_16 = 'https://github.com/Rhys-L/QOS_HTHH/blob/main/omi-so2/mask_OMI-Aura_L3-OMSO2e_ColumnAmountSO2_2022_1_16_8bit_calipso_rev2_lon_lat87.tif'
omi_mask_calipso_30 = 'https://github.com/Rhys-L/QOS_HTHH/blob/main/omi-so2/mask_OMI-Aura_L3-OMSO2e_ColumnAmountSO2_2022_1_30_8bit_calipso_rev2_constraint1297.tif'

### Load OMPS Tiles

In [18]:
omps_input_16 = os.path.join(omps_data,'clipped_OMPS-NPP_LP-L2-AER-DAILY_v2.1_2022m0116_2022m0512t115646.tif')
omps_input_30 = os.path.join(omps_data,'clipped_OMPS-NPP_LP-L2-AER-DAILY_v2.1_2022m0130_2022m0512t115721.tif',)

omps_mask_bright_16 = os.path.join(omps_data,'mask_OMPS-NPP_LP-L2-AER-DAILY_v2.1_2022m0116_brightest_multimask_false_36.tif')
omps_mask_bright_16_500 = os.path.join(omps_data,'mask_OMPS-NPP_LP-L2-AER-DAILY_v2.1_2022m0116_brightest_multimask_false_500.tif')

omps_mask_bright_30 = os.path.join(omps_data,'mask_OMPS-NPP_LP-L2-AER-DAILY_v2.1_2022m0130_brightest_multimask_false_36.tif')
omps_mask_bright_30_500 = os.path.join(omps_data,'mask_OMPS-NPP_LP-L2-AER-DAILY_v2.1_2022m0130_brightest_multimask_false_500.tif')


omps_mask_calipso_16 = os.path.join(omps_data,'mask_OMPS-NPP_LP-L2-AER-DAILY_v2.1_2022m0116_calipso_multimask_false_4.tif')
omps_mask_calipso_30 = os.path.join(omps_data,'mask_OMPS-NPP_LP-L2-AER-DAILY_v2.1_2022m0130_calipso_multimask_false_1297.tif')

### Prompt Brightest Pixels

In [19]:
def get_brightest_pixels(image_path, num_pixels=5):
    """
    Takes path to a raster image and count of x pixels to sample.
    Returns list of geo-coordinates of top x brightest pixels.
    List of coords can be used as input to segmentation model. 
    """
    # Open the TIFF image using rasterio
    with rasterio.open(image_path) as src:
         # Get the number of bands in the image and read data as numpy arr
        num_bands = src.count
        # Read the image data from the first band
        if num_bands == 1:
            image_data = src.read(1)
        elif num_bands == 2:
            # Read the image data from the second band to use so2 for training data
            image_data = src.read(2)
        else:
            raise ValueError("Image must have either 1 or 2 bands")

        # Find the indices of the pixels with the highest values
        flat_indices = np.argpartition(image_data.flatten(), -num_pixels)[-num_pixels:]
        max_indices = np.unravel_index(flat_indices, image_data.shape)
        max_indices = np.column_stack(max_indices)

        # Convert the pixel indices to coordinates using the image's geotransform

        coords = [(lon, lat) for lon, lat in [src.xy(row, col) for row, col in max_indices]]
        #print(coords)
        geometries = [Point(lat, lon) for lat, lon in coords]

        # Create a GeoDataFrame from the geometries
        gdf = gpd.GeoDataFrame(geometry=geometries, crs=src.crs)


    return gdf

In [20]:
omi_bright_prompt_16 = get_brightest_pixels(omi_input_16, 36)
omi_bright_prompt_30 = get_brightest_pixels(omi_input_30, 36)

In [21]:
omps_bright_prompt_16 = get_brightest_pixels(omps_input_16, 36)
omps_bright_prompt_30 = get_brightest_pixels(omps_input_30, 36)

### Create Map

In [30]:
### Instantiate Leafmap Object and add basemap

m = leafmap.Map(height="600px")
m.add_basemap("SATELLITE")

### Add point for Hunga Tonga Volcano
m.add_marker(location=[-20, -175], popup="Hunga Tonga", shape="circle", marker_colors="red")

### Add Calipso Prompt Points
#m.add_gdf(calipso_prompt_16, layer_name="CALIPSO Prompt Jan. 16")
#m.add_gdf(calipso_prompt_30, layer_name="CALIPSO Prompt Jan. 30")


### Add Raster Images as layers
m.add_raster(omi_input_16, layer_name="OMI Input Jan. 16", cmap='viridis') 
m.add_gdf(omi_bright_prompt_16, layer_name="OMI Prompt Bright Jan. 16")
m.add_raster(omi_mask_bright_16, layer_name="OMI Mask Bright Jan. 16")
m.add_raster(omi_mask_calipso_16, layer_name="OMI Mask CALIPSO Jan. 16")


m.add_raster(omi_input_30, layer_name="OMI Input Jan. 30", cmap='viridis')
m.add_gdf(omi_bright_prompt_30, layer_name="OMI Prompt Bright Jan. 30")
m.add_raster(omi_mask_bright_30, layer_name="OMI Mask Bright Jan. 30")
m.add_raster(omi_mask_calipso_30, layer_name="OMI CALIPSO Mask Jan. 30")


m.add_raster(omps_input_16, layer_name="OMPS Input Jan. 16", cmap='magma')
m.add_gdf(omps_bright_prompt_16, layer_name="OMPS Prompt Bright Jan. 16")
m.add_raster(omps_mask_bright_16, layer_name="OMPS Brightest Mask Jan. 16")
m.add_raster(omps_mask_calipso_16, layer_name="OMPS CALIPSO Mask Jan. 16")


m.add_raster(omps_input_30, layer_name="OMPS Input Jan. 30", cmap='magma')
m.add_gdf(omps_bright_prompt_30, layer_name="OMPS Prompt Bright Jan. 30")
m.add_raster(omps_mask_bright_30, layer_name="OMPS Brightest Mask Jan. 30")
m.add_raster(omps_mask_calipso_30, layer_name="OMPS CALIPSO Mask Jan. 30")

m.set_center(lat=-20, lon=-175, zoom=6)

m.add_layer_control()
m

### Write Map to HTML File

In [31]:
out_file = "demo.html"

# Write map to demo folder
m.to_html(out_file)